In [2]:
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [3]:
df = pd.read_csv('preprocessed_train.csv')
df.head()

,bant_submit,customer_country,business_unit,com_reg_ver_win_rate,customer_idx,customer_type,enterprise,customer_job,lead_desc_length,inquiry_type,product_category,product_subcategory,product_modelname,customer_country.1,customer_position,response_corporate,expected_timeline,ver_cus,ver_pro,ver_win_rate_x,ver_win_ratio_per_bu,business_area,lead_owner,is_converted,customer_job_category,product_category_status
0,1.0,/Quezon City/Philippines,AS,0.066667,32160,End-Customer,Enterprise,purchasing,62,Quotation or purchase consultation,multi-split,NaN,NaN,/Quezon City/Philippines,entry level,LGEPH,less than 3 months,1,0,0.003079,0.026846,corporate / office,0,True,purchasing,Filled
1,1.0,/PH-00/Philippines,AS,0.066667,23122,End-Customer,Enterprise,media and communication,96,Quotation or purchase consultation,multi-split,NaN,NaN,/PH-00/Philippines,ceo/founder,LGEPH,less than 3 months,1,0,0.003079,0.026846,corporate / office,1,True,media and communication,Filled
2,1.0,/Kolkata /India,AS,0.088889,1755,End-Customer,Enterprise,engineering,56,Product Information,single-split,NaN,NaN,/Kolkata /India,partner,LGEIL,less than 3 months,1,0,0.003079,0.026846,corporate / office,2,True,engineering,Filled
3,1.0,/Bhubaneswar/India,AS,0.088889,4919,End-Customer,Enterprise,entrepreneurship,44,Quotation or purchase consultation,vrf,NaN,NaN,/Bhubaneswar/India,ceo/founder,LGEIL,less than 3 months,1,0,0.003079,0.026846,corporate / office,3,True,entrepreneurship,Filled
4,1.0,/Hyderabad/India,AS,0.088889,17126,Specifier/ Influencer,Enterprise,consulting,97,Quotation or purchase consultation,multi-split,NaN,NaN,/Hyderabad/India,partner,LGEIL,less than 3 months,0,0,0.003079,0.026846,corporate / office,4,True,consulting,Filled


In [4]:
df.columns

Index(['bant_submit', 'customer_country', 'business_unit',
       'com_reg_ver_win_rate', 'customer_idx', 'customer_type', 'enterprise',
       'customer_job', 'lead_desc_length', 'inquiry_type', 'product_category',
       'product_subcategory', 'product_modelname', 'customer_country.1',
       'customer_position', 'response_corporate', 'expected_timeline',
       'ver_cus', 'ver_pro', 'ver_win_rate_x', 'ver_win_ratio_per_bu',
       'business_area', 'lead_owner', 'is_converted', 'customer_job_category',
       'product_category_status'],
      dtype='object')

In [5]:
for col in df.columns:
    if df[col].dtype != 'int64' and df[col].dtype != 'float64':
        print(col)

customer_country
business_unit
customer_type
enterprise
customer_job
inquiry_type
product_category
product_subcategory
product_modelname
customer_country.1
customer_position
response_corporate
expected_timeline
business_area
is_converted
customer_job_category
product_category_status


## customer_country

In [6]:
print(len(df.customer_country.unique()))
df.customer_country.unique()

15400


array(['/Quezon City/Philippines', '/PH-00/Philippines',
       '/Kolkata /India', ..., '/Pisco/Peru', '/santa cruz bolivia/Peru',
       '/paris/France'], dtype=object)

In [7]:
for unique in df.customer_country.unique():
    if len(df.customer_country.value_counts(unique)) > 100:
        print(unique)

/Quezon City/Philippines
/PH-00/Philippines
/Kolkata /India
/Bhubaneswar/India
/Hyderabad/India
/Abuja/Nigeria
/Jeddah, KSA/Saudi Arabia
/Guwahati/India
/Cebu city/Philippines
/hauz khas,delhi/India
/hosur/India
/Koduvai, Tiruppur/India
/Benin City/Nigeria
/Lagos/Nigeria
/Riyadh/Saudi Arabia
/Singapore/Singapore
//Philippines
/Cebu City/Philippines
/Noida/India
/BENGALURU/India
/chennai/India
/ghaziabad/India
/Pune/India
/KANPUR/India
/RS/Brazil
/Dubai/U.A.E
/Johannesburh/South Africa
/Posey/United States
/Bauan Batangas/Philippines
/Metro manila/Philippines
/Taguig/Philippines
/Mysore/India
/Kolkata/India
/Chennai/India
/Bangalore/India
/New Delhi/India
/Kaithal /India
/kozhikode/India
/Badarpur/India
/Gopalganj/India
/Mumbai /India
/MALDA/India
/COIMBATORE/India
/Aurangabad/India
/AHMEDABAD/India
/Santa Cruz do Sul - RS /Brazil
/Umm Al Quwain/U.A.E
/Centurion/South Africa
/Bogota/Colombia
/MedellÃ­n /Colombia
/Barrancabermeja/Colombia
/MedellÃ­n/Colombia
/Bucaramanga/Colombia
/Medell